In [ ]:
from qutip.tensor import tensor
from qutip import basis, sigmap, sigmam, qeye, sigmaz, sigmax, mesolve, mcsolve, sesolve
import matplotlib.pylab as plt
import numpy as np
import time
from qutip.qip.gates import (
    swap,
    gate_sequence_product,
    gate_expand_2toN,
    gate_expand_1toN,
)
from scipy.optimize import minimize as sp_minimize
from matplotlib.gridspec import GridSpec
import copy
from scipydirect import minimize as direct_minimize

In [ ]:
def H_Schwinger(N, w, m, g):
    """
    Creates a Qobj correspoding to the lattice Schwinger model (see: https://arxiv.org/pdf/1810.03421.pdf)
    Args:
        N (int): number of sites
        w (double): spin flip-flop coupling strength
        m (double): bare mass
        g (double): electric field coupling strength
    Returns:
        H (qobj): Qobj of N-site lattice Schwinger Hamiltonian.

    """
    H = 0
    identity_N = tensor([qeye(2)] * N)
    sigmapm = tensor([sigmap(), sigmam()])

    for i in range(N - 1):
        sigmapm_N = gate_expand_2toN(sigmapm, N, targets=[i, i + 1])
        H += w * (sigmapm_N + sigmapm_N.dag())

    Lzi = 0
    for i in range(N):
        sigmaz_N = gate_expand_1toN(sigmaz(), N, i)
        H += (m / 2) * ((-1) ** (i + 1)) * sigmaz_N
        Lzi += -(1 / 2) * (sigmaz_N + ((-1) ** (i + 1)) * identity_N)
        H += Lzi * Lzi * g
    return H

In [ ]:
def sigmaz_tot(N):
    """
    Args:
        N (int): number of sites
    Returns:
        sigmaz_tot (qobj): N-site total magnetic field operator
    """
    identity_N = tensor([qeye(2)] * N)
    _sigmaz_tot = 0
    for i in range(N):
        _sigmaz_tot += gate_expand_1toN(sigmaz(), N, i)
    return _sigmaz_tot


def CP(N):
    """
    Args:
        N (int): number of sites
    Returns:
        CP (qobj): N-site charge conjugation parity operator
    """
    identity_N = tensor([qeye(2)] * N)
    sigmax_sigmax = tensor([sigmax(), sigmax()])
    _cp = 1
    for i in range(N // 2):
        sigmax_sigmax_Ni = gate_expand_2toN(sigmax_sigmax, N, targets=[i, N - i - 1])
        _swap = swap(N=N, targets=[i, N - i - 1])
        _cp *= sigmax_sigmax_Ni * _swap
    return _cp

In [ ]:
def get_expectation_value(operator, psi):
    return ((psi.dag() * operator * psi)[0][0])[0]


def get_CP_quantum_number(psi, N):
    """
    Args:
        psi(qobj): quantum state
        N (int): number of sites
    Returns:
        E (int): quantum number corresponding to the charge conjugation operator
    """
    E = get_expectation_value(CP(N), psi)
    return E


def get_sigmaz_quantum_number(psi, N):
    """
    Args:
        psi(qobj): quantum state
        N (int): number of sites
    Returns:
        E (int): quantum number corresponding to the charge conjugation operator
    """
    E = get_expectation_value(sigmaz_tot(N), psi)
    return E

In [ ]:
mins = []
ms = np.linspace(-1.2, -0.2, 20)
ns = range(2, 11, 2)

### Energy gap vs bare mass

In [ ]:
%%time
plt.figure(figsize=(10, 8))
for n in ns:
    # print("qbit: ", n)
    t1 = time.time()
    dE = []
    for m in ms:
        H_T = H_Schwinger(n, 1.0, m, 1.0)
        energies = H_T.eigenenergies()
        dE.append(energies[1] - energies[0])
    # print("ground state: ", energies[0])
    # print("time taken: ", time.time()-t1)
    plt.plot(ms, dE, "-o", label="n = " + str(n))
    mins.append(min(dE))
plt.xlabel("Bare mass, m")
plt.ylabel("Energy gap, $\Delta$")
plt.legend()

In [ ]:
def H_resource_ent(N, B, J0, alpha):
    H_R = B * sigmaz_tot(N)
    sigmax_ij = tensor([sigmax(), sigmax()])
    for i in range(N - 1):
        for j in range(i + 1, N):
            sigmax_ij_N = gate_expand_2toN(sigmax_ij, N, targets=[i, j])
            H_R += (J0 / abs(i - j) ** alpha) * sigmax_ij_N
    return H_R


def H_resource_local(N, thetas):
    if N != len(thetas):
        print("length of theta list need to be equal to N")
    H_R = 0
    for ind, theta in enumerate(thetas):
        H_R += theta * H_resource_local_i(N, ind)
    return H_R


def H_resource_local_i(N, i):
    return gate_expand_1toN(sigmaz(), N, i)

In [ ]:
def get_ansatz(theta, Nb_layers, N, B, J0, alpha, psi_0, n_steps):
    """
    Args:
        theta (list<double>): list of variational parameters, the first parameter corresponds
                              to the time of the entangling gate which is assumed to be the
                              same for all odd layers and the remaining list of length
                              "N*Nb_of_layers/2" contains parameters for the even layers with
                              local unitaries.
        psi_0 (qobj): initial state.
    Returns:
        psi_1 (qobj): quantum state of the ansatz after evolving the state_in using
                      the variational parameter vector theta.
    """
    if len(theta) != (N // 2 + 1) * Nb_layers // 2:
        print("length: ", len(theta))
        print("Check theta vector")

    theta_in = list(theta)
    theta_final = []

    for i in range(Nb_layers // 2):
        theta_final += [theta_in[(N // 2 + 1) * i]]
        _temp = theta_in[(N // 2 + 1) * i + 1 : (N // 2 + 1) * i + 1 + N // 2]
        _temp_m = [-elm for elm in _temp]
        _temp = _temp + _temp_m[::-1]
        theta_final += _temp

    theta = theta_final

    if len(theta) != (N + 1) * Nb_layers // 2:
        print("length: ", len(theta))
        print("Check theta vector")

    for i in range(Nb_layers // 2):
        # psi_0 = (1j*theta[0]*H_resource_ent(N, B, J0, alpha, theta[0])).expm()*psi_0
        psi_0 = mesolve(
            H_resource_ent(N, B, J0, alpha),
            psi_0,
            np.linspace(0.0, theta[(N + 1) * i], n_steps),
            [],
            [],
        ).states[-1]
        # psi_0 = (1j*H_resource_local(N, theta[N*i+1:N*(i+1)+1])).expm()*psi_0
        for ind, _theta in enumerate(theta[(N + 1) * i + 1 : (N + 1) * i + 1 + N]):
            psi_0 = mesolve(
                H_resource_local_i(N, ind),
                psi_0,
                np.linspace(0.0, _theta, n_steps),
                [],
                [],
            ).states[-1]
    return psi_0

In [ ]:
##model params
N = 8
w = 1
m = 0.1
g = 1
##

## VQS params
Nb_layers = 4
# theta_in = list(np.random.rand((N+1)*Nb_layers//2))
theta_in = list(np.random.rand((N // 2 + 1) * Nb_layers // 2))

B = 10
J0 = 1
alpha = 1.34
psi_0 = tensor([basis(2, 0), basis(2, 1)] * (N // 2))
##

H_T = H_Schwinger(N, w, m, g)
gs = H_T.groundstate()
sigz_tot = sigmaz_tot(N)
energies = H_T.eigenenergies()

eigenstates = H_T.eigenstates()
e0 = eigenstates[0][0]

for i in range(1, 10):
    if np.abs(get_sigmaz_quantum_number(eigenstates[1][i], N)) == 0.0:
        e1 = eigenstates[0][i]
        print("intra sector gap: ", e1 - e0)
        break

print("symmetry checks")
print("ground state sigmaz_tot quantum number: ", get_sigmaz_quantum_number(gs[1], N))
print("ground state CP quantum number: ", get_CP_quantum_number(gs[1], N))
print("initial state sigmaz_tot quantum number: ", get_sigmaz_quantum_number(psi_0, N))
print("initial state CP quantum number: ", get_CP_quantum_number(psi_0, N))

In [ ]:
def cost(theta, H_T=H_T):
    psi_theta = get_ansatz(theta, Nb_layers, N, B, J0, alpha, psi_0, n_steps=2)
    res = np.real(get_expectation_value(H_T, psi_theta))
    fid = np.abs(psi_theta.dag() * gs[1]) ** 2
    epsilon = get_expectation_value((H_T - res * tensor([qeye(2)] * N)) ** 2, psi_theta)
    print(res, fid, epsilon)
    E0.append(res)
    fidelity.append(fid)
    epsilon_theta.append(epsilon)
    return res

In [ ]:
%%time
epsilon_theta = []
fidelity = []
E0 = []
bounds = [(0.0, 2.0 * np.pi) * len(theta_in)]
optimizer = sp_minimize(cost, theta_in, method="Nelder-Mead", options={"maxiter": 3000})

In [ ]:
print("Optimal theta = ", optimizer.x)
print("E_VQS = %s, E_GS =%s" % (optimizer.fun, gs[0]))

### Evolution of $E_{\theta}$, fidelity ($|\langle\psi_g|\psi(\theta)\rangle|^2$) and Hamiltonian variance($\langle(H_T -\langle H_T \rangle_{\theta})^2 \rangle_{\theta}$) as a function of iteration number

In [ ]:
plt.figure(figsize=(10, 3))

gspec = GridSpec(1, 3)
plt.subplot(gspec[0])
plt.plot(E0, "o")
plt.plot([gs[0]] * len(E0), label="$E_0$")
plt.xlabel("Iteration number")
plt.ylabel("E($\\theta$)")
# plt.yticks([1, 0, -1, -2, -3, -4 ])
# plt.ylim([-4.2, 1.5])
plt.legend()
plt.grid()

plt.subplot(gspec[1])
plt.plot([fid.data for fid in fidelity], "-")
plt.xlabel("Iteration number")
plt.ylabel("Fidelity")
plt.grid()

plt.subplot(gspec[2])
plt.plot(epsilon_theta, "-")
plt.xlabel("Iteration number")
plt.ylabel("Hamiltonian variance")
plt.grid()
plt.tight_layout()
plt.savefig(
    "VQS_schwinger_nspins_%s_Nblayers_%s_m_%s_g_%s.png" % (N, Nb_layers, m, g),
    bbox_inches="tight",
)

In [ ]:
n_iter = 650
plt.figure()
plt.plot(E0[:n_iter], "o", color="orange")
plt.plot([e0] * len(E0[:n_iter]), "--k", label="$E_0$")
plt.plot([e1] * len(E0[:n_iter]), "--", color="orange", label="$E_1$")
plt.xlabel("Iteration number i")
plt.ylabel("E($\\theta_i$)")
plt.yticks([4, 2, 0, -2, -4])
plt.ylim([-4.3, 4.3])
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(
    "Fig_3_VQS_schwinger_nspins_%s_Nblayers_%s_m_%s_g_%s_n_iter_%s.png"
    % (N, Nb_layers, m, g, n_iter)
)